In [2]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproductive

In [33]:
# paths
NOTE_DATA_DIR = '/local/XW/DATA/MIMIC/noteevents_by_sid/'
ICD_FPATH = 'data/subject_diag_icds.txt'
PK_FPATH = 'data/diag_processed_data.pk' # './processed_data_small.pk'
N_LABELS = 50

In [4]:
# read k-hot labels data
pk_data = pk.load(open(PK_FPATH, 'rb'))
Y_train = pk_data['Y_train']
Y_val = pk_data['Y_val']

In [5]:
pk_data.keys()

['embedding_matrix',
 'X_train',
 'X_val',
 'Y_val',
 'Y_train',
 'sid2khot',
 'sid2seq']

In [52]:
def multilabel_evaluate(y_pred, y_true=Y_val):
    tp = np.sum(y_true * y_pred, axis=-1)
    sum_true = np.sum(y_true, axis=-1)
    sum_pred = np.sum(y_pred, axis=-1)
    union = np.sum(np.clip(y_true+y_pred, 0, 1), axis=-1)
    print 'precision =', np.mean(tp/(sum_pred+1e-10))
    print 'recall = ', np.mean(tp/(sum_true+1e-10))
    print 'F1 = ', 2*np.mean(tp/(sum_true+sum_pred))
    print 'acc = ', np.mean(tp/union)

# Naive baseline

In [6]:
print Y_train.shape, Y_val.shape

(36917, 50) (9229, 50)


In [7]:
print Y_train[:2,]

[[ 1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]


In [10]:
# naive algo just find which label is most common, then always predict this label
stat = Y_train.sum(axis=0)
print stat

[ 14021.   8595.   8119.   7855.   6168.   5990.   5872.   5258.   4578.
   4558.   4390.   4220.   4196.   3952.   3483.   3409.   3296.   3015.
   2882.   2810.   2829.   2718.   2670.   2539.   2362.   2304.   2348.
   2286.   2225.   2198.   2226.   2214.   2175.   2128.   2028.   2020.
   1897.   1852.   1854.   1866.   1847.   1815.   1787.   1713.   1642.
   1642.   1638.   1591.   1599.  35142.]


In [15]:
pred_naive = stat > Y_train.shape[0]/2
pred_naive = np.array(pred_naive, dtype=np.float64)
print pred_naive
pred_naive = np.tile(pred_naive, (Y_val.shape[0],1))
print pred_naive.shape

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
(9229, 50)


In [23]:
multilabel_evaluate(pred_naive)

precision = 0.952107487173
recall =  0.265610235902
F1 =  0.378717632882
acc =  0.265610235902


# SVM baseline

** The problem with SVM is that for each instance, there are several labels, so I split each instance into several (instance, one-label) pairs... NOT SURE IF THIS MAKES SENSE. **

This baseline use the method described at: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [5]:
# prepare label data
N_LABELS = 50
K_ICDS_TOKEEP = N_LABELS - 1 # predict only on top K frequent icd codes
N_SUBJECTS = 46146
from collections import Counter
sid2icds = {} # map subject_id ---> icd codes of this patient
icd_ctr = Counter()
with open(ICD_FPATH) as f: 
    for line in tqdm(f, total=N_SUBJECTS): 
        sid, _icds = line.split(',')
        _icds = _icds.split()
        icd_ctr.update(_icds)
        sid2icds[sid] = set(_icds)

100%|██████████| 46146/46146 [00:00<00:00, 81862.25it/s]


In [6]:
print icd_ctr.most_common(K_ICDS_TOKEEP)
icds = zip( *icd_ctr.most_common(K_ICDS_TOKEEP) )[0] + ('other',)
sid2khot = {} # map subject_id to k-hot vector
for sid in sid2icds.keys():
    _khot = np.zeros(N_LABELS)
    for _icd in sid2icds[sid]:
        if _icd in icds: 
            _khot[icds.index(_icd)] = 1
        else: # label 'other icds'
            _khot[-1] = 1
    sid2khot[sid] = _khot

[('4019', 17510), ('41401', 10736), ('42731', 10193), ('4280', 9802), ('5849', 7634), ('2724', 7421), ('25000', 7332), ('51881', 6632), ('5990', 5746), ('V053', 5678), ('V290', 5440), ('2720', 5320), ('53081', 5246), ('2859', 4967), ('486', 4391), ('2851', 4231), ('2762', 4120), ('2449', 3789), ('496', 3572), ('99592', 3504), ('V3000', 3503), ('0389', 3387), ('5070', 3362), ('V5861', 3184), ('3051', 2982), ('311', 2907), ('41071', 2902), ('5859', 2889), ('40390', 2814), ('2761', 2789), ('2875', 2783), ('412', 2775), ('V3001', 2707), ('4240', 2643), ('5119', 2554), ('V1582', 2534), ('78552', 2376), ('V4581', 2318), ('4241', 2302), ('9971', 2299), ('42789', 2297), ('V4582', 2247), ('7742', 2241), ('5845', 2154), ('2760', 2077), ('5180', 2072), ('45829', 2055), ('V5867', 2009), ('V502', 1978)]


In [7]:
sids = sid2icds.keys()
np.random.shuffle(sids)
VALIDATION_SPLIT = 0.2
validset_sz = int(VALIDATION_SPLIT*len(sids))
train_sids = sids[:-validset_sz] 
val_sids = sids[-validset_sz:]

In [8]:
Y_train = np.array([sid2khot[sid] for sid in train_sids])
Y_val = np.array([sid2khot[sid] for sid in val_sids])

In [9]:
Y_train.shape, Y_val.shape

((36917, 50), (9229, 50))

**Train 50 independent SVM classifiers**.   
To prepare training data for svm, need to write our own **generator** of documents. 

In [10]:
def notes_generator(fpaths):
    for fpath in tqdm(fpaths):
        df = pd.read_csv(fpath)
        yield '\n=======\n\n\n'.join(df['text'])

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
MAX_NB_WORDS = 20000 # top 20k most freq words
cnt_vect = CountVectorizer(max_features=MAX_NB_WORDS)

In [12]:
train_files = [os.path.join(NOTE_DATA_DIR, '%s.csv'%sid) for sid in train_sids]
X_train_counts = cnt_vect.fit_transform(notes_generator(train_files))
print X_train_counts.shape
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

100%|██████████| 36917/36917 [09:01<00:00, 68.24it/s]


(36917, 20000)
(36917, 20000)


In [13]:
val_files = [os.path.join(NOTE_DATA_DIR, '%s.csv'%sid) for sid in val_sids]
X_val_counts = cnt_vect.fit_transform(notes_generator(val_files))
print X_val_counts.shape
tfidf_transformer = TfidfTransformer()
X_val_tfidf = tfidf_transformer.fit_transform(X_val_counts)
print X_val_tfidf.shape

100%|██████████| 9229/9229 [03:02<00:00, 50.49it/s]


(9229, 20000)
(9229, 20000)


In [14]:
print X_train_tfidf.shape, Y_train.shape

(36917, 20000) (36917, 50)


** generate 0/1 label for each class ** 

In [15]:
Y_train_labels = [Y_train[:,i] for i in xrange(Y_train.shape[1])]

In [33]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
clfs = []
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=1e-3, n_iter=200, random_state=1)
    clf.fit(X_train_tfidf, Y_train_labels[i]) 
    clfs.append(clf)

100%|██████████| 50/50 [21:22<00:00, 21.17s/it]


** predict and evaluate **

In [34]:
preds = [clfs[i].predict(X_val_tfidf) for i in xrange(N_LABELS)]    
pred_svm = np.vstack(preds).T
print pred_svm.shape

(9229, 50)


In [35]:
print pred_svm[:10]

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0

In [36]:
multilabel_evaluate(pred_svm)

precision = 0.609780763494
recall =  0.27759335575
F1 =  0.335145619096
acc =  0.219569032631


In [38]:
clfs2 = []
for i in tqdm(range(N_LABELS)):
#     clf = SVC(class_weight='balanced',random_state=1)
    clf =  SGDClassifier(loss='hinge', penalty='l2', 
                   class_weight='balanced',
                   alpha=5e-4, n_iter=200, random_state=1)
    clf.fit(X_train_tfidf, Y_train_labels[i]) 
    clfs2.append(clf)
preds = [clfs2[i].predict(X_val_tfidf) for i in xrange(N_LABELS)]    
pred_svm2 = np.vstack(preds).T
print pred_svm2.shape
multilabel_evaluate(pred_svm2)

100%|██████████| 50/50 [21:06<00:00, 21.40s/it]


(9229, 50)
precision = 0.934626358848
recall =  0.263854597276
F1 =  0.374515908556
acc =  0.262168750158


** dump file for later use ** 

In [21]:
data_svm = {
    'X_train': X_train_tfidf,
    'Y_train': Y_train,
    'X_val': X_val_tfidf,
    'Y_val': Y_val,
    'train_sids': train_sids,
    'val_sids': val_sids
}

In [22]:
pk.dump(data_svm, open('./diag_data_svm_new.pk','wb'), pk.HIGHEST_PROTOCOL)

# Deep patient baseline

In [25]:
VALIDATION_SPLIT = 0.2

In [6]:
sid2khot = pk_data['sid2khot'] # maps sid(int) to khot encoding of Y_true

In [11]:
sid2khot[2]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [15]:
data = pk.load(open('./data/deep_patient_vec.pk'))
dpvecs = data['dpvecs']
sid2rowidx = data['sid2rowidx'] # map sid(int) to row index in deep patient feature matrix

In [17]:
len(sid2rowidx), len(sid2khot)

(46146, 46146)

In [24]:
sids = np.array(sid2rowidx.keys())
np.random.shuffle(sids)
print sids

[32441 69537 19904 ..., 21795 12247 10963]


In [26]:
sid_train, sid_val = sids[int(len(sids)*VALIDATION_SPLIT):], sids[:int(len(sids)*VALIDATION_SPLIT)]

In [27]:
len(sid_train), len(sid_val)

(36917, 9229)

In [30]:
X_train, Y_train = [],[]
for sid in sid_train:
    X_train.append(dpvecs[sid2rowidx[sid]])
    Y_train.append(sid2khot[sid])
X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)
print X_train.shape, Y_train.shape

X_val, Y_val = [],[]
for sid in sid_val:
    X_val.append(dpvecs[sid2rowidx[sid]])
    Y_val.append(sid2khot[sid])
X_val = np.vstack(X_val)
Y_val = np.vstack(Y_val)
print X_val.shape, Y_val.shape

(36917, 500) (36917, 50)
(9229, 500) (9229, 50)


In [60]:
Y_train.sum(axis=0)

array([ 14020.,   8628.,   8187.,   7810.,   6091.,   5945.,   5861.,
         5292.,   4610.,   4541.,   4394.,   4234.,   4227.,   3952.,
         3530.,   3368.,   3278.,   3034.,   2817.,   2817.,   2773.,
         2715.,   2710.,   2524.,   2397.,   2293.,   2355.,   2303.,
         2267.,   2214.,   2219.,   2194.,   2190.,   2124.,   2062.,
         2044.,   1919.,   1839.,   1870.,   1865.,   1833.,   1803.,
         1818.,   1743.,   1640.,   1639.,   1651.,   1608.,   1588.,
        35142.])

In [61]:
Y_val.sum(axis=0)

array([ 3490.,  2108.,  2006.,  1992.,  1543.,  1476.,  1471.,  1340.,
        1136.,  1137.,  1046.,  1086.,  1019.,  1015.,   861.,   863.,
         842.,   755.,   755.,   687.,   730.,   672.,   652.,   660.,
         585.,   614.,   547.,   586.,   547.,   575.,   564.,   581.,
         517.,   519.,   492.,   490.,   457.,   479.,   432.,   434.,
         464.,   444.,   423.,   411.,   437.,   433.,   404.,   401.,
         390.,  8787.])

In [69]:
print map(int, pred_svm_dp.sum(axis=0))

[63, 0, 0, 0, 768, 0, 0, 3117, 2824, 88, 347, 0, 26, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9229]


In [70]:
clfs = []
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
for i in tqdm(range(N_LABELS)):
    clf = SVC(class_weight='balanced',random_state=1)
#     clf =  SGDClassifier(loss='hinge', penalty='l2', 
# #                    class_weight='balanced',
#                    alpha=1e-2, n_iter=200, random_state=1)
    clf.fit(X_train, Y_train[:,i]) 
    clfs.append(clf)

100%|██████████| 50/50 [8:45:24<00:00, 467.98s/it]


In [67]:
preds = [clfs[i].predict(X_val) for i in xrange(N_LABELS)]    
pred_svm_dp = np.vstack(preds).T
print pred_svm_dp.shape
multilabel_evaluate(y_pred=pred_svm_dp,y_true=Y_val)

(9229, 50)
precision = 0.785121175938
recall =  0.301476883694
F1 =  0.392286694427
acc =  0.272933425004
